# Decoding with SpaceNet  -- Milkshake Data
https://nilearn.github.io/modules/generated/nilearn.decoding.SpaceNetClassifier.html#nilearn.decoding.SpaceNetClassifier

In [2]:
#this imports all the commands needed for the script to work#
import os
import numpy as np
import nilearn
import glob
#import matplotlib
import nibabel as nib
import pandas as pd 
from nilearn.input_data import NiftiMasker 
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Agg')
from nilearn.image import index_img

### Load our Chocolate Milkshake data
- Starting with W1 (134 subjects) 
- We will start with decoding LF_HS vs h2O 

In [ ]:
#image mask
imag_mask='/projects/niblab/bids_projects/Experiments/ChocoData/images/power_roimask_4bi.nii.gz'
#our behavioral csv file 
stim = '/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/w1_milkshake_100.csv'
#our dataset concatenated image 
dataset='/projects/niblab/bids_projects/Experiments/ChocoData/images/w1_milkshake_100.nii.gz'
behavioral = pd.read_csv(stim, sep="\t")
#grab conditional labels 
conditions = behavioral["Label"]
condition_mask = behavioral["Label"].isin(['LF_HS_receipt', "h20_receipt"])
conditions = conditions[condition_mask]

condition_mask_train = (condition_mask) & (behavioral['sess'] == 0)
condition_mask_test = (condition_mask) & (behavioral['sess']  == 1)

# Apply this sample mask to X (fMRI data) and y (behavioral labels)
# Because the data is in one single large 4D image, we need to use
# index_img to do the split easily
X_train = index_img(dataset, condition_mask_train)
X_test = index_img(dataset, condition_mask_test)
y_train = conditions[condition_mask_train]
y_test = conditions[condition_mask_test]

# Compute the mean epi to be used for the background of the plotting
from nilearn.image import mean_img
background_img = mean_img(dataset)

In [ ]:
# fit space net with graph-net penalty 
from nilearn.decoding import SpaceNetClassifier
decoder = SpaceNetClassifier(memory="nilearn_cache", penalty="graph-net")
decoder.fit(X_train, y_train)
y_pred = decoder.predict(X_test)
accuracy=y(y_pred==y_test).mean() * 100.
print("Graph-net classification accuracy : %g%% " % accuracy)

In [ ]:
from nilearn.plotting import plot_stat_map

coef_img = decoder.coef_img_
plot_stat_map(coef_img, background_img, title="graph-net: accuracy ")